In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import regex as re
from lxml import etree as et
from itertools import repeat
import csv
import random
import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
header_list = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,                           like Gecko) Chrome/103.0.5060.66 Safari/537.36",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393"]

base_url = 'https://www.backloggd.com/games/lib/popular?page='
games_list = []
d = 0

In [3]:
game_links = []

for page_no in tqdm_notebook(range(10)):
    page_url = base_url + str(page_no)
    #print(page_url)
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(page_url, headers = header)
    if webpage.status_code == 200:
        soup1 = BeautifulSoup(webpage.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
        g = soup2.find('div', {'class' : 'row show-release toggle-fade'})
        games = g.find_all('div', {'class' : 'col-2 my-2 px-1 px-md-2'})
        game_ref = [game.find('a').get('href') for game in games]
        game_links.extend(['https://www.backloggd.com' + ref for ref in game_ref])

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
print(len(game_links))

324


In [5]:
cols = ['Title', 'Release Date', 'Team', 'Rating', 'Times Listed', 'Number of Reviews', 'Genres', 'Summary', 'Reviews', 'Plays', 'Playing', 'Backlogs', 'Wishlist']
backloggd = pd.DataFrame(columns = cols)

In [6]:
for link in tqdm_notebook(game_links):
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(link, headers = header)
    if webpage.status_code == 200:
        soup1 = BeautifulSoup(webpage.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
        
        title = soup2.find('div', {'class' : 'col-auto pr-1'}).get_text().strip()
        
        release_date = ' '.join(soup2.find('div', {'class' : 'col-auto mt-auto pr-0'}).get_text().strip().split()[-3:])

        try:
            teams = soup2.find('div', {'class' : 'col-auto pl-lg-1 sub-title'})
            teams = teams.find_all('a')
            teams = [i.get_text().strip() for i in teams]
        except:
            teams = np.nan
            #print(teams)
        
        try:
            rating = float(soup2.find(id = 'score').get_text().strip()[-3:])
        except:
            rating = np.nan

        table = soup2.find_all('div', {'class' : 'col-12 mb-1'})#.get_text().strip()
        feats = [f.get_text().strip().split('\n')[0].strip() for f in table]
        results = [r.get_text().strip().split('\n')[-1].strip() for r in table]
        #print(feats,results)

        # dicted = {}
        # for i in range(len(feats)):
        #     dicted[feats[i]] = results[i]
        
        nlists = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[0]
        
        nreviews = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[0]

        genres = soup2.find_all('p',{'class' : 'genre-tag'})
        genres = [genre.get_text().strip() for genre in genres]

        summary = soup2.find(id = 'collapseSummary').get_text().strip()

        review_section = soup2.find(id = 'game-reviews-section')
        reviews = review_section.find_all('div', {'class' : 'row pt-2 pb-1 review-card'})
        reviews = [r.find('div', {'class' : 'formatted-text'}).get_text().strip() for r in reviews]

        row = [title, release_date, teams, rating, nlists, nreviews, genres, summary, reviews]
        row.extend(results)
        backloggd.loc[len(backloggd.index)] = row

  0%|          | 0/324 [00:00<?, ?it/s]

In [9]:
backloggd.head(20)

,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,Elden Ring,"Feb 25, 2022","[FromSoftware, Bandai Namco Entertainment]",4.5,6.2K,6.2K,"[Adventure, RPG]","Elden Ring is a fantasy, action and open world...","[👍, A good effort from konami, but at the end ...",33K,5.2K,8.7K,7.3K
1,The Legend of Zelda: Tears of the Kingdom,"May 12, 2023","[Nintendo, Nintendo EPD Production Group No. 3]",4.5,4.2K,4.2K,"[Adventure, RPG]",The Legend of Zelda: Tears of the Kingdom is t...,"[nice addition to the first game, played only ...",16K,6.5K,6.3K,6.9K
2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","[Nintendo EPD Production Group No. 3, Nintendo]",4.4,6.6K,6.6K,"[Adventure, Puzzle, RPG]",The Legend of Zelda: Breath of the Wild is the...,"[👍, very nice game, my first Legend of Zelda, ...",48K,3.7K,7.6K,3.9K
3,Hades,"Dec 07, 2018",[Supergiant Games],4.3,4.2K,4.2K,"[Adventure, Brawler, Indie, RPG]",A rogue-lite hack and slash dungeon crawler in...,"[I enjoyed it quite a bit, but with games like...",35K,4.4K,10K,5.3K
4,Hollow Knight,"Feb 24, 2017",[Team Cherry],4.4,4.6K,4.6K,"[Adventure, Indie, Platform]",A 2D metroidvania with an emphasis on close co...,"[didnt got the true ending, thats a hard metro...",35K,3.6K,13K,3.6K
5,Undertale,"Sep 15, 2015","[tobyfox, 8-4]",4.3,5.1K,5.1K,"[Adventure, Indie, RPG, Turn Based Strategy]","A small child falls into the Underground, wher...","[One of if not the best indie game, An extreme...",43K,984,8.2K,2.9K
6,Resident Evil 4,"Mar 24, 2023","[Capcom, Capcom Development Division 1]",4.5,3.5K,3.5K,"[Adventure, Puzzle, Shooter]",Resident Evil 4 is a remake of the 2005 origin...,[played it like 4 times becuase how aweome thi...,13K,1.3K,4.3K,5.6K
7,Minecraft,"Nov 18, 2011",[Mojang Studios],4.3,3.5K,3.5K,"[Adventure, Simulator]",Minecraft focuses on allowing the player to ex...,"[👍, This has to be one of my favorite games of...",53K,3.1K,1.8K,323
8,Red Dead Redemption 2,"Oct 26, 2018","[Take-Two Interactive, Rockstar Games]",4.5,4.7K,4.7K,"[Adventure, RPG, Shooter]",Red Dead Redemption 2 is the epic tale of outl...,"[depressed howdy, O MAIOR JA FEITO, I cried in...",33K,3.1K,10K,4.8K
9,Baldur's Gate 3,"Oct 06, 2020",[Larian Studios],4.6,1.8K,1.8K,"[Adventure, RPG, Strategy, Tactical, Turn Base...","An ancient evil has returned to Baldur's Gate,...","[Jogaço, um dos melhores que já joguei na vida...",7K,5.3K,4.4K,5.6K


In [ ]:
backloggd.to_csv('backloggd.csv')